In [1]:
import torch
import numpy as np

In [2]:
torch.__version__

'1.9.0'

# `view()`, `reshape()`, `transpose()`, `permute()`


[Discussion of `contiguous`](https://stackoverflow.com/questions/26998223/what-is-the-difference-between-contiguous-and-non-contiguous-arrays/26999092#26999092)

In [3]:
sample_tensor = torch.rand(12)

In [4]:
sample_tensor

tensor([0.6640, 0.1038, 0.8594, 0.0016, 0.4127, 0.3789, 0.9644, 0.5370, 0.7852,
        0.1875, 0.8188, 0.5665])

In [5]:
sample_tensor.view(3, 4)

tensor([[0.6640, 0.1038, 0.8594, 0.0016],
        [0.4127, 0.3789, 0.9644, 0.5370],
        [0.7852, 0.1875, 0.8188, 0.5665]])

In [6]:
sample_tensor.reshape(3, 4)

tensor([[0.6640, 0.1038, 0.8594, 0.0016],
        [0.4127, 0.3789, 0.9644, 0.5370],
        [0.7852, 0.1875, 0.8188, 0.5665]])

## `reshape()` vs `view()`

`reshape` tries to return a `view` if possible, otherwise copies to data to a contiguous tensor and returns the `view` on it. ([Source](https://discuss.pytorch.org/t/difference-between-view-reshape-and-permute/54157/2))

In [7]:
x = torch.arange(12).view(4, 3)

# View works on contiguous tensors
print(x.is_contiguous())
print(x)
print(x.view(-1))

True
tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])


In [8]:
y = x.permute(1, 0)

# Reshape works on non-contugous tensors (contiguous() + view)
print(y)
print(y.is_contiguous())

tensor([[ 0,  3,  6,  9],
        [ 1,  4,  7, 10],
        [ 2,  5,  8, 11]])
False


In [9]:
try:
    print(y.view(-1))
except RuntimeError as e:
    print(e)

view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.


In [10]:
print(y.reshape(-1))
print(y.contiguous().view(-1))

tensor([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11])
tensor([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11])


The usage of `view` and `reshape` does not depend on training / not-training.
I personally use `view` whenever possible and add a `contiguous` call to it, if necessary. This will make sure I see, where a copy is done in my code. `reshape` on the other hand does this automatically, so your code might look cleaner. [Source](https://discuss.pytorch.org/t/difference-between-view-reshape-and-permute/54157/4)

## `transpose()` vs `permute()`

`permute()` and `tranpose()` are similar. `transpose()` can only swap two dimension. But `permute()` can swap all the dimensions.

In [11]:
x = torch.rand(2, 3, 4)

In [12]:
x, x.shape

(tensor([[[0.2460, 0.6898, 0.8167, 0.2560],
          [0.9936, 0.1473, 0.3890, 0.9937],
          [0.3012, 0.7747, 0.6782, 0.2940]],
 
         [[0.3836, 0.8078, 0.2072, 0.3382],
          [0.2485, 0.3106, 0.3912, 0.7263],
          [0.4162, 0.8130, 0.6254, 0.5643]]]),
 torch.Size([2, 3, 4]))

In [13]:
x.transpose(0, 2), x.transpose(0, 2).shape

(tensor([[[0.2460, 0.3836],
          [0.9936, 0.2485],
          [0.3012, 0.4162]],
 
         [[0.6898, 0.8078],
          [0.1473, 0.3106],
          [0.7747, 0.8130]],
 
         [[0.8167, 0.2072],
          [0.3890, 0.3912],
          [0.6782, 0.6254]],
 
         [[0.2560, 0.3382],
          [0.9937, 0.7263],
          [0.2940, 0.5643]]]),
 torch.Size([4, 3, 2]))

In [14]:
x.permute(2, 1, 0).shape

torch.Size([4, 3, 2])

In [15]:
(x.transpose(0, 2) == x.permute(2, 1, 0)).all()

tensor(True)

In [16]:
(x.transpose(0, 2) == x.transpose(2, 0)).all()

tensor(True)

Note that, in `permute()`, you must provide the new order of all the dimensions. In `transpose()`, you can only provide two dimensions. `tranpose()` can be thought as a special case of `permute()` method in for 2D tensors.